## Tune LSTM parameters

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns

In [3]:
import rpy2.robjects as robjects

folder_name = '3_state_model'

data_log = pd.read_csv(folder_name + "/close_data_log.csv")

# Helper function to convert R data.frame to pandas DataFrame
def r_to_df(r_df):
    return pd.DataFrame({name: np.array(r_df.rx2(name)) for name in r_df.names})

# Read the RDS files using rpy2
readRDS = robjects.r['readRDS']

hmm_stats_r = readRDS(folder_name + "/hmm_stats_df.rds")
hmm_stats_df = r_to_df(hmm_stats_r)

hmm_state_losses = np.array(readRDS(folder_name + "/hmm_state_losses.rds"))
hmm_predictions = np.array(readRDS(folder_name + "/hmm_predictions.rds"))
hmm_ground_truth = np.array(readRDS(folder_name + "/hmm_ground_truth.rds"))

X_train = np.array(readRDS(folder_name + "/lstm_X_train.rds"))
Y_train = np.array(readRDS(folder_name + "/lstm_Y_train.rds"))
X_valid = np.array(readRDS(folder_name + "/lstm_X_valid.rds"))
Y_valid = np.array(readRDS(folder_name + "/lstm_Y_valid.rds"))
X_test = np.array(readRDS(folder_name + "/lstm_X_test.rds"))
Y_test = np.array(readRDS(folder_name + "/lstm_Y_test.rds"))

n_states = Y_train.shape[2]

In [5]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
import keras_tuner as kt

# Assuming X_train, Y_train, X_valid, Y_valid, and n_states are defined elsewhere
# lookback and forecast_horizon are provided but not directly used in model building

# Compute Dirichlet parameters and predicted probabilities (for reference, not used in model)
def dirichlet_layer(evidence):
    alpha = evidence + 1
    S = tf.reduce_sum(alpha, axis=-1, keepdims=True)
    probs = alpha / S
    return probs, alpha, S

# Custom evidential loss with annealing
class EvidentialLoss(tf.keras.losses.Loss):
    def __init__(self, annealing_rate=100.0, name='evidential_loss'):
        super().__init__(name=name)
        self.annealing_rate = tf.constant(annealing_rate, dtype=tf.float32)
        self.current_epoch = tf.Variable(0.0, trainable=False, dtype=tf.float32)
    
    def call(self, y_true, evidence):
        alpha = evidence + 1
        S = tf.reduce_sum(alpha, axis=-1, keepdims=True)
        probs = alpha / S
        err = tf.square(y_true - probs)
        var = probs * (1 - probs) / (S + 1)
        
        # Annealing factor for the regularization term
        lambda_ = tf.minimum(1.0, self.current_epoch / self.annealing_rate)
        
        return tf.reduce_mean(tf.reduce_sum(err + lambda_ * var, axis=-1))

# Custom callback to update the current epoch in the loss instance
class AnnealingCallback(tf.keras.callbacks.Callback):
    def on_epoch_begin(self, epoch, logs=None):
        if hasattr(self.model, 'loss') and isinstance(self.model.loss, EvidentialLoss):
            self.model.loss.current_epoch.assign(tf.cast(epoch, tf.float32))

2026-01-26 15:57:00.042970: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2026-01-26 15:57:10.873698: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1769471831.863282 3631216 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1769471832.225040 3631216 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1769471834.331838 3631216 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

In [9]:
X_train[0, -5:, 0:3]

array([[0.00000000e+00, 0.00000000e+00, 1.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 1.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 1.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 1.00000000e+00],
       [0.00000000e+00, 1.00000000e+00, 2.94924502e-13]])

## Run hypertuning

In [ ]:
forecast_horizon = 0  # Example horizon index for validation

# Model-building function for hyperparameter tuning
def build_model(hp):
    
    # Tune lookback (history length/timesteps)
    lookback = hp.Choice('lookback', values=[3, 5, 10])
    
    # Tune feature inclusion (one boolean per group of 4 columns)
    include_tsx = hp.Boolean('include_TSX_Composite', default=True)
    include_crude = hp.Boolean('include_Crude_Oil_Futures', default=True)
    include_gold = hp.Boolean('include_Gold_Futures', default=True)
    include_copper = hp.Boolean('include_Copper_Futures', default=True)
    include_sp500 = hp.Boolean('include_SP_500', default=True)
    
    # Build feature mask: 1.0 for included groups, 0.0 for excluded
    feature_mask_list = (
        [1.0] * 4 if include_tsx else [0.0] * 4
    ) + (
        [1.0] * 4 if include_crude else [0.0] * 4
    ) + (
        [1.0] * 4 if include_gold else [0.0] * 4
    ) + (
        [1.0] * 4 if include_copper else [0.0] * 4
    ) + (
        [1.0] * 4 if include_sp500 else [0.0] * 4
    )
    feature_mask = tf.constant(feature_mask_list, dtype=tf.float32)
    
    # Other hyperparameters (unchanged)
    lstm_units = hp.Choice('lstm_units', values=[16, 32])
    dense_units = hp.Choice('dense_units', values=[8, 16])
    dropout_rate = 0.3 # hp.Float('dropout_rate', min_value=0.1, max_value=0.5, step=0.1)
    #learning_rate = hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='log')

    model = models.Sequential([
        layers.Input(shape=X_train.shape[1:]),  # Fixed shape (10, 20)
        
        # Lambda layer to slice to the last 'lookback' timesteps
        layers.Lambda(lambda x: x[:, -lookback:, :]),
        
        # Lambda layer to apply feature mask (broadcast to (batch, timesteps, 20))
        layers.Lambda(lambda x: x * tf.reshape(feature_mask, (1, 1, 20))),
        
        # LSTM layer with added dropout
        layers.LSTM(lstm_units, dropout=dropout_rate),
        
        layers.Dense(dense_units, activation='relu'),
        layers.Dropout(dropout_rate),
        layers.Dense(dense_units, activation='relu'),
        layers.Dropout(dropout_rate),
        # Output raw evidence (non-negative)
        layers.Dense(n_states, activation='softplus')
    ])
    
    model.compile(
        loss=EvidentialLoss(annealing_rate=100.0),
        optimizer='adam',
        metrics=['accuracy']  # Accuracy on argmax(probs)
    )
    
    return model

# Initialize the tuner (using Hyperband for efficiency)
tuner = kt.Hyperband(
    build_model,
    objective='val_loss',  # Optimize for validation loss
    max_epochs=75,
    factor=3,
    directory='tuner_dir',
    project_name='evidential_lstm_tuning_v2'
)

# Define callbacks: annealing and early stopping
annealing_callback = AnnealingCallback()
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Perform the hyperparameter search
tuner.search(
    X_train, 
    Y_train[:, forecast_horizon, :], 
    epochs=75, 
    validation_data=(X_valid, Y_valid[:, forecast_horizon, :]),
    callbacks=[annealing_callback, early_stopping],
    verbose=1  # Adjust verbosity as needed
)

# Retrieve the best model and hyperparameters
best_model = tuner.get_best_models(num_models=1)[0]
best_hyperparameters = tuner.get_best_hyperparameters(num_trials=3)[0]

# Optionally, print the best hyperparameters
print(best_hyperparameters.values)

# Summary of the best model
best_model.summary()

Trial 90 Complete [00h 00m 08s]
val_loss: 0.22197426855564117

Best val_loss So Far: 0.2187802791595459
Total elapsed time: 00h 05m 38s
{'lookback': 3, 'include_TSX_Composite': True, 'include_Crude_Oil_Futures': False, 'include_Gold_Futures': True, 'include_Copper_Futures': False, 'include_SP_500': True, 'lstm_units': 32, 'dense_units': 16, 'tuner/epochs': 25, 'tuner/initial_epoch': 9, 'tuner/bracket': 3, 'tuner/round': 2, 'tuner/trial_id': '0041'}


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lambda (Lambda)                 │ (None, 3, 20)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lambda_1 (Lambda)               │ (None, 3, 20)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 32)             │         6,784 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 16)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 16)             │           272 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 16)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 3)              │            51 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 7,635 (29.82 KB)

 Trainable params: 7,635 (29.82 KB)

 Non-trainable params: 0 (0.00 B)

In [15]:
tuner.get_best_hyperparameters(num_trials=3)[0].values

{'lookback': 3,
 'include_TSX_Composite': True,
 'include_Crude_Oil_Futures': False,
 'include_Gold_Futures': True,
 'include_Copper_Futures': False,
 'include_SP_500': True,
 'lstm_units': 32,
 'dense_units': 16,
 'tuner/epochs': 25,
 'tuner/initial_epoch': 9,
 'tuner/bracket': 3,
 'tuner/round': 2,
 'tuner/trial_id': '0041'}

## Run tuning for all lookback periods

In [18]:
pd.DataFrame(best_params_list)

,lookback,include_TSX_Composite,include_Crude_Oil_Futures,include_Gold_Futures,include_Copper_Futures,include_SP_500,lstm_units,dense_units,tuner/epochs,tuner/initial_epoch,tuner/bracket,tuner/round,tuner/trial_id,horizon
0,3,True,True,True,False,True,32,16,75,25,3,3,0047,0
1,10,False,True,True,True,True,16,16,75,25,3,3,0049,1
2,5,True,True,False,False,True,32,16,25,9,2,1,0061,2
3,3,True,False,False,True,True,32,16,25,9,2,1,0058,3
4,10,False,True,False,True,False,32,8,9,3,3,1,0013,4
5,3,False,False,False,True,False,32,16,9,3,3,1,0008,5


In [16]:
# List to collect best hyperparameters for each horizon
best_params_list = []

# Loop through forecast horizons from 0 to 9
for horizon in range(10):
    print(f"Starting hyperparameter tuning for forecast horizon: {horizon}")
    
    # Initialize a new tuner for each horizon to avoid state conflicts
    tuner = kt.Hyperband(
        build_model,
        objective='val_loss',  # Optimize for validation loss
        max_epochs=75,
        factor=3,
        directory='tuner_dir',
        project_name=f'evidential_lstm_tuning_h{horizon}_v2'  # Unique project name per horizon
    )

    # Define callbacks: annealing and early stopping
    annealing_callback = AnnealingCallback()
    early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

    # Perform the hyperparameter search for this horizon
    tuner.search(
        X_train, 
        Y_train[:, horizon, :], 
        epochs=75, 
        validation_data=(X_valid, Y_valid[:, horizon, :]),
        callbacks=[annealing_callback, early_stopping],
        verbose=1  # Adjust verbosity as needed
    )

    # Retrieve the best hyperparameters for this horizon
    best_hyperparameters = tuner.get_best_hyperparameters(num_trials=1)[0]
    
    # Create a dictionary of the values and include the horizon
    params_dict = best_hyperparameters.values.copy()
    params_dict['horizon'] = horizon
    
    # Append to the list
    best_params_list.append(params_dict)

# Convert the list of dictionaries to a Pandas DataFrame
best_params_df = pd.DataFrame(best_params_list)

# Optionally, print the DataFrame
print(best_params_df)

Trial 23 Complete [00h 00m 13s]
val_loss: 0.40580233931541443

Best val_loss So Far: 0.3067939281463623
Total elapsed time: 00h 04m 49s


ResourceExhaustedError: tuner_dir/evidential_lstm_tuning_h6_v2/trial_0023/trial.json; Disk quota exceeded